# The Model module

The top module handles the main setup of the instance. 


- Requires:

    - The data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be >= 0.6
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.model as pm

### Graphics

In [ ]:
import pyposeidon.utils.pplot # initialize matplolib accessor

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
import pyposeidon.utils.hplot # initialize pyviz accessor

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

In [ ]:
import numpy
cpath = numpy.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

## Setup

The minimum information required is the geometry's extent. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In addition, the coastlines need to be provided as internal boundaries. Without loss of generality, natural earth's 'intermediate' resolution (https://www.naturalearthdata.com/downloads/) is used here.

The coastlines can be given as a shapefile or GeoDataFrame.

In [ ]:
# use cartopy to get coastlines
import cartopy.feature as cf

cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

In [ ]:
# observations: get SEASET data
import pandas as pd
seaset = pd.read_csv("https://raw.githubusercontent.com/tomsail/seaset/main/Notebooks/catalog_full.csv",
                     index_col=0)
seaset.to_csv("data/seaset_full.csv")
seaset.head()

In [ ]:
#define in a dictionary the properties of the model..
model={'solver_name':'telemac',
     'geometry':{'lon_min' : -25,'lon_max' : -12.,'lat_min' : 56.,'lat_max' : 74.},
     'coastlines':ne_i,  # coastlines
#     'manning':.12, #uniform manning value
#     'windrot':0.00001, #uniform windrot value
#     'tag':'schism', # optional tag 
     'mesh_generator' : 'jigsaw', # set mesh generator  
     'start_date':'2017-10-1 0:0:0',
     'time_frame':'12H',
     'meteo_source' : ['./data/erai.grib'], #path to meteo files
     'dem_source' : './data/dem.nc', #path to dem file
#     'ncores': 4 , # optional number of cores
     'dt' : 50,
     'rpath':'./test/telemac/', #location of calc folder
#     'update':['all'], #set which component should be updated  (meteo,dem,model)
    'monitor':True, # get time series for observation points
    'obs' : "data/seaset_full.csv",
    }

## Initialize

In [ ]:
#initialize a model
model['mesh_generator'] = 'jigsaw'
b = pm.set(**model)

## Step by Step process

In [ ]:
b.create()
b.mesh.Dataset.pplot.mesh()

TPXO tide have been implemented, but have not been tested in pyposeidon.

We will show here a simple case of surge model, without tide forcing at the boundaries

In [ ]:
b.mesh.Dataset.type[:] = 'closed' # here for a surge application, we will close all boundaries

In [ ]:
b.output()

In [ ]:
b.save() # saves the json model file

In [ ]:
b.set_obs() # setup station points

In [ ]:
b.run() # execute

## ... or simply in one step


In [ ]:
#initialize a model
b = pm.set(**model)
b.execute()

The various datasets incorporated in the model can be accessed independently as attributes, namely :

- Meteo

In [ ]:
#plt.figure() # xarray uses existing ax if any. You might need to create a new figure to do it separately.
b.meteo.Dataset.msl[2,:,:].plot()

- Mesh

In [ ]:
b.mesh.Dataset.hplot.mesh() 

- Dem

In [ ]:
b.dem.Dataset.elevation.plot()

- Coastlines

In [ ]:
b.coastlines.plot()

- Boundaries

In [ ]:
b.mesh.boundary.show()

## Output

The output of the simulations could be in separate files (due to MPI) or specific folders. pyposeidon can incorporate the resulting Datasets with 

In [ ]:
b.results(remove_zarr=False) # integrate output

using [xarray-selafin](https://github.com/seareport/xarray-selafin/) directly on the selafin files: 

In [ ]:
import xarray as xr
out = xr.open_dataset(b.rpath + '/results_2D.slf', engine = 'selafin', lazy_loading = False)
out

Or using xarray on the generated results: 

In [ ]:
! tar -xvf test/telemac/outputs/out_2D.zarr.tar

In [ ]:
out = xr.open_dataset('out_2D.zarr')
out

In [ ]:
out.pplot.mesh(x='longitude', y = 'latitude', e='face_nodes')

In [ ]:
it = 2
u, v = out.u.isel(time = it).values, out.v.isel(time = it).values
out.pplot.quiver(x='longitude', y = 'latitude', it = 2, u=u, v=v, scale=.5, title='hvel', color='k')

### with hvplot

In [ ]:
out.hplot.contourf(var='depth')#,tiles=True)

In [ ]:
out.hplot.mesh(width=800, height=400,tiles=True)

### Animate

In [ ]:
import numpy as np 
v = out.pplot.frames(x='longitude', 
                     y = 'latitude', 
                     e='face_nodes',
                     var='elev',
                     title='SSH', 
                     vmin = -.5, 
                     vmax =.5) # change to hplot for holoviews
v

In [ ]:
v.save('test/elev.mp4')

In [ ]:
u, v = out.u.values, out.v.values
out.pplot.qframes(x='longitude', 
                     y = 'latitude', 
                     e='face_nodes',
                     u=u,v=v,title='Vel',scale=.5)#, color='w')

## Read Model

If a model is created by `pyposeidon` then there is a `json` file that describes the model. Such a file can be read into `pyposeidon` with 

In [ ]:
a = pyposeidon.model.read('test/telemac/telemac2d_model.json')

In [ ]:
a.execute()

However, there might be a model created by other means. `pyposeidon` does its best to incorporate such model with

TELEMAC Works up until here 

`read_folder()` class function is yet to be implemented

In [ ]:
c = pm.set(solver_name='telemac', rfolder = './test/telemac/', load_geo=True, load_meteo=True)

In [ ]:
c.mesh.Dataset

In [ ]:
c.meteo.Dataset